In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
url2 = 'https://user.wartale.com/index.asp?page=30'
response2 = requests.get(url2)
soup = BeautifulSoup(response2.content, 'html.parser')

In [3]:
data = []
span = soup.find('span')
for a_tag in span.find_all('a'):
    data.append({
        'id_type': a_tag.get('id'),
        'category': a_tag.text
    })

df = pd.DataFrame(data)

df['subtypes'] = df['id_type'].apply(lambda x: f'sub{x}')

df

,id_type,category,subtypes
0,type_1,Weapons,subtype_1
1,type_2,Defenses,subtype_2
2,type_3,Accessories,subtype_3


In [4]:
subtypes = df['subtypes'].tolist()
subtypes

['subtype_1', 'subtype_2', 'subtype_3']

In [5]:
data2 = []
for stype in subtypes:
    a = soup.find('span', id = stype)
    for a_tag in a.find_all('a'):
        data2.append({
            'subtypes': stype,
            'id_subtype': a_tag.get('id'),
            'type': a_tag.text
        })
df2 = pd.DataFrame(data2)    
df2["items"] = df2['id_subtype'].str.replace('sub_', 'items_')
df2    

,subtypes,id_subtype,type,items
0,subtype_1,sub_2,Axes,items_2
1,subtype_1,sub_5,Bows,items_5
2,subtype_1,sub_4,Claws,items_4
3,subtype_1,sub_19,Daggers,items_19
4,subtype_1,sub_21,Fists,items_21
5,subtype_1,sub_7,Hammers,items_7
6,subtype_1,sub_6,Javelins,items_6
7,subtype_1,sub_20,Phantoms,items_20
8,subtype_1,sub_8,Scythes,items_8
9,subtype_1,sub_3,Swords,items_3


In [6]:
Table1 = df2[['items', 'type']]	
Table1	

,items,type
0,items_2,Axes
1,items_5,Bows
2,items_4,Claws
3,items_19,Daggers
4,items_21,Fists
5,items_7,Hammers
6,items_6,Javelins
7,items_20,Phantoms
8,items_8,Scythes
9,items_3,Swords


In [10]:
import sqlite3
conn = sqlite3.connect('ItemsDB.db')
Table1.to_sql('items', conn, if_exists='replace', index=False)
conn.close()



In [11]:
items = df2['items'].tolist()
items

['items_2',
 'items_5',
 'items_4',
 'items_19',
 'items_21',
 'items_7',
 'items_6',
 'items_20',
 'items_8',
 'items_3',
 'items_1',
 'items_10',
 'items_9',
 'items_11',
 'items_12',
 'items_13',
 'items_14',
 'items_15',
 'items_16',
 'items_23',
 'items_22',
 'items_17',
 'items_18']

In [12]:
data3 = []
for item in items:
    c = soup.find('div', id=item)
    for i in c.find_all('tr'):
        try:
            data3.append({
                "items" : item,
                'onClick': i.td['onclick'],
                'title': i.img['title'],
                'item_name': i.span.text
            })
        except KeyError:
            pass
        except TypeError:
            pass

df3 = pd.DataFrame(data3)
df3   

,items,onClick,title,item_name
0,items_2,"loadItem2(627, 1);",1-handed,Stone Axe
1,items_2,"loadItem2(628, 1);",1-handed,Steel Axe
2,items_2,"loadItem2(629, 1);",1-handed,Battle Axe
3,items_2,"loadItem2(630, 1);",1-handed,War Axe
4,items_2,"loadItem2(631, 1);",2-handed,Double Sided War Axe
...,...,...,...,...
370,items_1,"loadItem2(4913, 1);",2-handed,Mythology Staff
371,items_1,"loadItem2(11064, 1);",2-handed,Corrupted Staff
372,items_1,"loadItem2(17164, 1);",1-handed,Eternal Wand
373,items_1,"loadItem2(19228, 1);",2-handed,Rapture Staff


In [13]:
df3['item_id'] = df3['onClick'].str.extract(r'loadItem2\((\d+), \d+\);')
df3

,items,onClick,title,item_name,item_id
0,items_2,"loadItem2(627, 1);",1-handed,Stone Axe,627
1,items_2,"loadItem2(628, 1);",1-handed,Steel Axe,628
2,items_2,"loadItem2(629, 1);",1-handed,Battle Axe,629
3,items_2,"loadItem2(630, 1);",1-handed,War Axe,630
4,items_2,"loadItem2(631, 1);",2-handed,Double Sided War Axe,631
...,...,...,...,...,...
370,items_1,"loadItem2(4913, 1);",2-handed,Mythology Staff,4913
371,items_1,"loadItem2(11064, 1);",2-handed,Corrupted Staff,11064
372,items_1,"loadItem2(17164, 1);",1-handed,Eternal Wand,17164
373,items_1,"loadItem2(19228, 1);",2-handed,Rapture Staff,19228


In [70]:
# Make a request to the website
url = 'https://user.wartale.com/guide.asp?page=1&action=2&id=627'
response = requests.get(url)
soupitem = BeautifulSoup(response.content, 'html.parser')


In [87]:
itemsid = df3['item_id'].tolist()
itemsid

['627',
 '628',
 '629',
 '630',
 '631',
 '632',
 '633',
 '634',
 '635',
 '636',
 '637',
 '638',
 '639',
 '640',
 '641',
 '642',
 '643',
 '644',
 '645',
 '646',
 '650',
 '651',
 '647',
 '652',
 '648',
 '653',
 '654',
 '649',
 '655',
 '892',
 '4909',
 '11056',
 '17165',
 '19225',
 '20783',
 '771',
 '772',
 '773',
 '774',
 '775',
 '776',
 '777',
 '778',
 '779',
 '780',
 '781',
 '782',
 '783',
 '784',
 '785',
 '786',
 '787',
 '788',
 '789',
 '790',
 '791',
 '792',
 '822',
 '823',
 '793',
 '824',
 '794',
 '825',
 '826',
 '795',
 '827',
 '896',
 '4911',
 '11063',
 '17168',
 '19230',
 '20795',
 '656',
 '657',
 '658',
 '659',
 '660',
 '661',
 '662',
 '663',
 '664',
 '665',
 '666',
 '667',
 '668',
 '669',
 '670',
 '671',
 '672',
 '673',
 '674',
 '675',
 '679',
 '676',
 '680',
 '677',
 '681',
 '682',
 '678',
 '683',
 '893',
 '4915',
 '11062',
 '17167',
 '19226',
 '20784',
 '8992',
 '8993',
 '8995',
 '8994',
 '8996',
 '8997',
 '8998',
 '8999',
 '9000',
 '9001',
 '9002',
 '9003',
 '9004',
 '9005',

In [69]:
def extract_values(tr):
    left_values = tr.select_one('td.item_box_text_left').get_text('\n', strip=True).split('\n')
    right_values = tr.select_one('td:not(.item_box_text_left)').get_text('\n', strip=True).split('\n')
    
    data_dict = {}
    for left, right in zip(left_values, right_values):
        data_dict[left] = right
    return data_dict



# Extract data from the webpage
item_box_header = soupitem.find('div', class_='item_box_header')
item_name = item_box_header.contents[0].strip()
item_id = soupitem.find("div", class_="item_box_img").img["alt"]
item_image_url = soupitem.find("div", class_="item_box_img").img["src"]

tr_indices = [1, 10]
data = {}
data = {
    "item_name": item_name,
    "item_id": item_id,
    "item_image_url": item_image_url
}

for index in tr_indices:
    tr = soupitem.find_all('tr')[index]
    data.update(extract_values(tr))
    
    # Extract price and weight values
    price_weight_values = tr.select_one('td:not(.item_box_text_left)').get_text('\n', strip=True).split('\n')
    data["weightxd"] = price_weight_values[-2]
    data["pricexd"] = price_weight_values[-1]

# Convert data to a Pandas DataFrame
df2 = pd.DataFrame([data])


# Print the DataFrame
df2

,item_name,item_id,item_image_url,Level,Strength,Talent,weightxd,pricexd,Primary Spec,Secondary Spec,Weight,Price
0,Memorial Axe,WA310,images/items/itWA310.jpg,146,360,95,142,3920000,FS,MS,PS,142


In [80]:
import pandas as pd

def extract_values(tr):
    left_values = tr.select_one('td.item_box_text_left').get_text('\n', strip=True).split('\n')
    right_values = tr.select_one('td:not(.item_box_text_left)').get_text('\n', strip=True).split('\n')
    
    data_dict = {}
    for left, right in zip(left_values, right_values):
        data_dict[left] = right
    return data_dict

# Make a request to the website
url = 'https://user.wartale.com/guide.asp?page=1&action=2&id=20783'
response = requests.get(url)
soupitem = BeautifulSoup(response.content, 'html.parser')

# Extract data from the webpage
item_box_header = soupitem.find('div', class_='item_box_header')
item_name = item_box_header.contents[0].strip()
item_id = soupitem.find("div", class_="item_box_img").img["alt"]
item_image_url = soupitem.find("div", class_="item_box_img").img["src"]

tr_indices = [1, 10]
data = {
    "item_name": item_name,
    "item_id": item_id,
    "item_image_url": item_image_url
}

for index in tr_indices:
    tr = soupitem.find_all('tr')[index]
    data.update(extract_values(tr))

# Convert data to a Pandas DataFrame
df = pd.DataFrame([data])

# Print the DataFrame
df

,item_name,item_id,item_image_url,Level,Strength,Talent,Primary Spec,Secondary Spec,Weight,Price
0,Memorial Axe,WA310,images/items/itWA310.jpg,146,360,95,FS,MS,PS,142


In [81]:
import pandas as pd

def extract_values(tr):
    left_values = tr.select_one('td.item_box_text_left').get_text('\n', strip=True).split('\n')
    right_values = tr.select_one('td:not(.item_box_text_left)').get_text('\n', strip=True).split('\n')
    
    data_dict = {}
    for left, right in zip(left_values, right_values):
        data_dict[left] = right
    return data_dict

# Extract data from the webpage
item_box_header = soupitem.find('div', class_='item_box_header')
item_name = item_box_header.contents[0].strip()
item_id = soupitem.find("div", class_="item_box_img").img["alt"]
item_image_url = soupitem.find("div", class_="item_box_img").img["src"]

tr_indices = [1, 10]
data = {
    "item_name": item_name,
    "item_id": item_id,
    "item_image_url": item_image_url
}

for index in tr_indices:
    try:
        tr = soupitem.find_all('tr')[index]
        data.update(extract_values(tr))
        
        # Extract price and weight values
        price_weight_values = tr.select_one('td:not(.item_box_text_left)').get_text('\n', strip=True).split('\n')
        data["weightxd"] = price_weight_values[-2]
        data["pricexd"] = price_weight_values[-1]
    except IndexError:
        pass

# Convert data to a Pandas DataFrame
df2 = pd.DataFrame([data])

df2

,item_name,item_id,item_image_url,Level,Strength,Talent,weightxd,pricexd,Primary Spec,Secondary Spec,Weight,Price
0,Memorial Axe,WA310,images/items/itWA310.jpg,146,360,95,142,3920000,FS,MS,PS,142


In [82]:
xd1 = pd.concat([df, df2], ignore_index=True)
xd1

,item_name,item_id,item_image_url,Level,Strength,Talent,Primary Spec,Secondary Spec,Weight,Price,weightxd,pricexd
0,Memorial Axe,WA310,images/items/itWA310.jpg,146,360,95,FS,MS,PS,142,NaN,NaN
1,Memorial Axe,WA310,images/items/itWA310.jpg,146,360,95,FS,MS,PS,142,142,3920000


In [ ]:


for item in itemsid:
    url = 'https://user.wartale.com/guide.asp?page=1&action=2&id={item}'
    def extract_values(tr):
    left_values = tr.select_one('td.item_box_text_left').get_text('\n', strip=True).split('\n')
    right_values = tr.select_one('td:not(.item_box_text_left)').get_text('\n', strip=True).split('\n')
    
    data_dict = {}
    for left, right in zip(left_values, right_values):
        data_dict[left] = right
    return data_dict

# Extract data from the webpage
item_box_header = soupitem.find('div', class_='item_box_header')
item_name = item_box_header.contents[0].strip()
item_id = soupitem.find("div", class_="item_box_img").img["alt"]
item_image_url = soupitem.find("div", class_="item_box_img").img["src"]

tr_indices = [1, 10]
data = {
    "item_name": item_name,
    "item_id": item_id,
    "item_image_url": item_image_url
}

for index in tr_indices:
    try:
        tr = soupitem.find_all('tr')[index]
        data.update(extract_values(tr))
        
        # Extract price and weight values
        price_weight_values = tr.select_one('td:not(.item_box_text_left)').get_text('\n', strip=True).split('\n')
        data["weightxd"] = price_weight_values[-2]
        data["pricexd"] = price_weight_values[-1]
    except IndexError:
        pass
    
    

In [95]:
itemsid = df3['item_id'].tolist()

import time 
def extract_values(tr):
    left_values = tr.select_one('td.item_box_text_left').get_text('\n', strip=True).split('\n')
    right_values = tr.select_one('td:not(.item_box_text_left)').get_text('\n', strip=True).split('\n')
    
    data_dict = {}
    for left, right in zip(left_values, right_values):
        data_dict[left] = right
    return data_dict

xd1 = pd.DataFrame()  # Create an empty DataFrame to store the extracted data

for item in itemsid:
    url = f'https://user.wartale.com/guide.asp?page=1&action=2&id={item}'
    response = requests.get(url)
    soupitem = BeautifulSoup(response.content, 'html.parser')
    
    item_box_header = soupitem.find('div', class_='item_box_header')
    item_name = item_box_header.contents[0].strip()
    item_id = soupitem.find("div", class_="item_box_img").img["alt"]
    item_image_url = soupitem.find("div", class_="item_box_img").img["src"]
    
    tr_indices = [1, 10]
    data = {
        "xd": item,
        "item_name": item_name,
        "item_id": item_id,
        "item_image_url": item_image_url
    }
    
    for index in tr_indices:
        try:
            tr = soupitem.find_all('tr')[index]
            data.update(extract_values(tr))
            
            # Extract price and weight values
            price_weight_values = tr.select_one('td:not(.item_box_text_left)').get_text('\n', strip=True).split('\n')
            data["weightxd"] = price_weight_values[-2]
            data["pricexd"] = price_weight_values[-1]
        except IndexError:
            pass
    
    df = pd.DataFrame([data])
    xd1 = pd.concat([xd1, df], ignore_index=True)
    time.sleep(2)
    print(xd1)

    xd  item_name item_id            item_image_url   Primary Spec  \
0  627  Stone Axe   WA101  images/items/itWA101.jpg             FS   

  Secondary Spec Weight Price weightxd pricexd  
0             MS      5    60        5      60  
    xd  item_name item_id            item_image_url   Primary Spec  \
0  627  Stone Axe   WA101  images/items/itWA101.jpg             FS   
1  628  Steel Axe   WA102  images/items/itWA102.jpg             FS   

  Secondary Spec Weight Price weightxd pricexd  
0             MS      5    60        5      60  
1             MS    ASS     8        8     420  
    xd   item_name item_id            item_image_url      Primary Spec  \
0  627   Stone Axe   WA101  images/items/itWA101.jpg                FS   
1  628   Steel Axe   WA102  images/items/itWA102.jpg                FS   
2  629  Battle Axe   WA103  images/items/itWA103.jpg  NaN           FS   

  Secondary Spec Weight Price weightxd pricexd Level Strength  
0             MS      5    60        5    

KeyboardInterrupt: 

In [93]:
xd1

,xd,item_name,item_id,item_image_url,,Primary Spec,Secondary Spec,Weight,Price,weightxd,pricexd
0,627,Stone Axe,WA101,images/items/itWA101.jpg,,FS,MS,5,60,5,60
1,628,Steel Axe,WA102,images/items/itWA102.jpg,,FS,MS,ASS,8,8,420
